In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [6]:
# config
config = {
    "data_path": "../",
    "model": {
        "encoder_name": "timm-resnest26d",
        "loss_smooth": 1.0,
        "optimizer_params": {"lr": 0.005, "weight_decay": 0.0},
        "scheduler": {
            "name": "CosineAnnealingLR",
            "params": {
                "CosineAnnealingLR": {"T_max": 2, "eta_min": 1e-06, "last_epoch": -1},
                "ReduceLROnPlateau": {
                    "factor": 0.316,
                    "mode": "min",
                    "patience": 3,
                    "verbose": True,
                },
            },
        },
        "seg_model": "Unet",
    },
    "output_dir": "models",
    "progress_bar_refresh_rate": 10,
    "seed": 42,
    "train_bs": 128,
    "use_aug": True,
    "trainer": {
        "enable_progress_bar": True,
        "max_epochs": 100,
        "min_epochs": 80,
        "accelerator": "mps",
        "devices": 1,
    },
    "valid_bs": 128,
    "workers": 0,
    "device": "mps",
    "folds": {
        "n_splits": 4,
        "random_state": 42,
        "train_folds": [0, 1, 2, 3]
    }
}


In [7]:
import torch
import numpy as np
import torchvision.transforms as T

import torch
import numpy as np
import torchvision.transforms as T

class ContrailsDataset(torch.utils.data.Dataset):
    def __init__(self, df, image_size=256, train=True, use_augmentations=None):

        self.df = df
        self.trn = train
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.image_size = image_size
        self.use_augmentations = use_augmentations
        if image_size != 256:
            self.resize_image = T.transforms.Resize(image_size)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        con_path = row.path
        con = np.load(str(con_path))

        img = con[..., :-1]
        label = con[..., -1]

        if self.image_size != 256:
            img = self.resize_image(img)

        if self.use_augmentations:
            transformed = self.use_augmentations(image = img, mask = label)
            img = transformed["image"]
            label = transformed["mask"]

        
        label = torch.tensor(label)

        img = torch.tensor(np.reshape(img, (256, 256, 3))).to(torch.float32).permute(2, 0, 1)

        img = self.normalize_image(img)

        return img.float(), label.float()

    def __len__(self):
        return len(self.df)


In [8]:
# Lightning module

import torch
import lightning.pytorch as pl
import segmentation_models_pytorch as smp
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from torch.optim import AdamW
import torch.nn as nn
from torchmetrics.functional import dice, f1_score, jaccard_index
# from torchmetrics import IoU


seg_models = {
    "Unet": smp.Unet,
    "Unet++": smp.UnetPlusPlus,
    "MAnet": smp.MAnet,
    "Linknet": smp.Linknet,
    "FPN": smp.FPN,
    "PSPNet": smp.PSPNet,
    "PAN": smp.PAN,
    "DeepLabV3": smp.DeepLabV3,
    "DeepLabV3+": smp.DeepLabV3Plus,
}

class DiceLoss(nn.Module):
    def __init__(self, smooth=1):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, logits, targets):
        logits = torch.sigmoid(logits)

        # flatten label and prediction tensors
        logits = logits.view(-1)
        targets = targets.view(-1)

        intersection = (logits * targets).sum()
        return 1 - (2.0 * intersection + self.smooth) / (
            logits.sum() + targets.sum() + self.smooth
        )


class LightningModule(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = model = seg_models[config["seg_model"]](
            encoder_name=config["encoder_name"],
            encoder_weights=None,
            in_channels=3,
            classes=1,
            activation=None,
        )
        # Load the pre-trained weights into the model, excluding the final layer
        state_dict = torch.load('models/Unet_timm-resnest26d.ckpt')
        # del state_dict['model.classifier.weight']  # Adjust these keys based on your model's structure
        # del state_dict['model.classifier.bias']
        self.model.load_state_dict(state_dict, strict=False)

        self.loss_module = DiceLoss(smooth=config["loss_smooth"])
        self.val_step_outputs = []
        self.val_step_labels = []

    def forward(self, batch):
        imgs = batch
        return self.model(imgs)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), **self.config["optimizer_params"])

        if self.config["scheduler"]["name"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(
                optimizer,
                **self.config["scheduler"]["params"][self.config["scheduler"]["name"]],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        elif self.config["scheduler"]["name"] == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer,
                **self.config["scheduler"]["params"][self.config["scheduler"]["name"]],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        labels = labels.unsqueeze(1)
        preds = self.model(imgs)
        loss = self.loss_module(preds, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=config["train_bs"])

        for param_group in self.trainer.optimizers[0].param_groups:
            lr = param_group["lr"]
        self.log("lr", lr, on_step=True, on_epoch=False, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        labels = labels.unsqueeze(1)
        preds = self.model(imgs)
        loss = self.loss_module(preds, labels)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        # self.val_step_outputs.append(preds.detach().cpu())
        # self.val_step_labels.append(labels.detach().cpu())

    # def on_validation_epoch_end(self):
    #     all_preds = torch.cat(self.val_step_outputs)
    #     all_labels = torch.cat(self.val_step_labels)
    #     self.val_step_outputs.clear()
    #     self.val_step_labels.clear()
        # val_dice = dice(all_preds, all_labels.long())
        # val_f1 = f1_score(all_preds.sigmoid(), all_labels.long(), task = "binary")
        # val_iou = jaccard_index(num_classes=2, task='binary', preds=all_preds.sigmoid(), target=all_labels.long())
        # self.log("val_iou", val_iou, on_step=False, on_epoch=True, prog_bar=True)

In [9]:
import warnings
import gc

warnings.filterwarnings("ignore")

import os
import torch
import pandas as pd
import lightning.pytorch as pl
from pprint import pprint
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
from torch import mps
import albumentations as A

contrails = os.path.join(config["data_path"], "contrails/")
data_path = os.path.join(config["data_path"], "data_df.csv")

data_df = pd.read_csv(data_path)

kf = KFold(n_splits=config["folds"]["n_splits"], shuffle=True, random_state=config["seed"])

if config["use_aug"] == True:
    transform_set = A.Compose([A.VerticalFlip(p=0.5),
                            A.HorizontalFlip(p=0.5),
                            A.RandomRotate90(p=0.5)])
else:
    transform_set = None

for fold, (train_index, valid_index) in enumerate(kf.split(data_df)):
    train_df = data_df.iloc[train_index]
    valid_df = data_df.iloc[valid_index]

    dataset_train = ContrailsDataset(train_df, train=True, use_augmentations=transform_set)
    dataset_validation = ContrailsDataset(valid_df, train=False)

    data_loader_train = DataLoader(
        dataset_train, batch_size=config["train_bs"], shuffle=True, num_workers=config["workers"]
    )
    data_loader_validation = DataLoader(
        dataset_validation, batch_size=config["valid_bs"], shuffle=False, num_workers=config["workers"]
    )

    pl.seed_everything(config["seed"])

    filename = f"{config['model']['seg_model']}_{config['model']['encoder_name']}_contrails_aug_{config['trainer']['min_epochs']}_{fold}"

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath=config["output_dir"],
        mode="min",
        filename=filename,
        save_top_k=1,
        verbose=1,
    )

    progress_bar_callback = TQDMProgressBar(refresh_rate=config["progress_bar_refresh_rate"])

    early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=1)

    trainer = pl.Trainer(
        callbacks=[checkpoint_callback, early_stop_callback, progress_bar_callback], logger=None, **config["trainer"]
    )

    model = LightningModule(config["model"])

    trainer.fit(model, data_loader_train, data_loader_validation)
    torch.save(model.state_dict(), os.path.join(config["output_dir"], f"{filename}.pth"))
    model = model.to('cpu')
    mps.empty_cache()
    del (
        dataset_train,
        dataset_validation,
        train_df,
        valid_df,
        data_loader_train,
        data_loader_validation,
        model,
        trainer,
        checkpoint_callback,
        progress_bar_callback,
        early_stop_callback,
    )
    mps.empty_cache()
    gc.collect()

Global seed set to 42


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name        | Type     | Params
-----------------------------------------
0 | model       | Unet     | 24.0 M
1 | loss_module | DiceLoss | 0     
-----------------------------------------
24.0 M    Trainable params
0         Non-trainable params
24.0 M    Total params
96.134    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Monitored metric val_loss did not improve in the last 14 records. Best score: 0.378. Signaling Trainer to stop.
Epoch 77, global step 10296: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 15 records. Best score: 0.378. Signaling Trainer to stop.
Epoch 78, global step 10428: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Metric val_loss improved by 0.001 >= min_delta = 0.0. New best score: 0.378
Epoch 79, global step 10560: 'val_loss' reached 0.37792 (best 0.37792), saving model to '/Users/johnny/Library/CloudStorage/OneDrive-Personal/py/Kaggle/contrails/notebooks/models/Unet_timm-resnest26d_contrails_aug_80_3.ckpt' as top 1


In [6]:
# save the model
torch.save(model.state_dict(), 'models/model.pt')

NameError: name 'model' is not defined

In [ ]:
data_df = pd.concat([train_df, valid_df])
data_df = data_df.reset_index(drop=True)
data_df.to_csv("../data_df.csv", index=False)

In [ ]:
torch.mps.driver_allocated_memory()